In [0]:
%run "./startup" 

In [0]:
params = WidgetParams({
    "sr_folio": str
})
# Validar widgets
params.validate()

In [0]:
conf = ConfManager()

#Archivos SQL
query = QueryManager()

#Conexion a base de datos
db = DBXConnectionManager()

In [0]:
#Query Extrae informacion para realiza el update de apagado de vigencia

statement_upd = query.get_statement(
    "PATRIF_AIN_0100_ACT_IND_MARCA_MEJORA_UPDATE_DSM_FOV.sql",
    sr_folio=params.sr_folio
)

db.write_delta(f"DELTA_TRA_MEJORA_UPDATE_{params.sr_folio}", db.read_data("default", statement_upd), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_MEJORA_UPDATE_{params.sr_folio}"))

In [0]:
#Query Extrae informacion para realizar el insert con el nuevo registro corregido 

statement_ins02 = query.get_statement(
    "PATRIF_AIN_0100_ACT_IND_MARCA_MEJORA_INSERT_DSM_FOV.sql",
    sr_folio=params.sr_folio
)

db.write_delta(f"DELTA_TRA_MEJORA_INSERT02_{params.sr_folio}", db.read_data("default", statement_ins02), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_MEJORA_INSERT02_{params.sr_folio}"))

In [0]:
# Cargar tabla Delta en un DataFrame
df_upd = spark.read.format("delta").table(f"DELTA_TRA_MEJORA_UPDATE_{params.sr_folio}")
df_ins02 = spark.read.format("delta").table(f"DELTA_TRA_MEJORA_INSERT02_{params.sr_folio}")

# Mostrar los primeros 100 registros
#df.show(100)

In [0]:
#INSERTA EN LA TABLA AUXILIAR

table_name_MARCA_DESMARCA = "CIERREN_DATAUX.TTAFOGRAL_IND_CTA_INDV_AUX"

db.write_data(db.read_delta(f"DELTA_TRA_MEJORA_UPDATE_{params.sr_folio}"), table_name_MARCA_DESMARCA, "default", "append")

In [0]:
#Se ejecuta el merge del Update


statement = query.get_statement(
    "PATRIF_AIN_0100_ACT_IND_MARCA_MEJORA_MERGE_DSM_FOV.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#Se eliminan los registros de la tabla auxiliar
statement = query.get_statement(
    "PATRIF_AIN_0100_ACT_IND_MARCA_MEJORA_DELETE_AUX.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#INSERTA EN LA TABLA CIERREN.TTAFOGRAL_IND_CTA_INDV

table_name_CTA_INDV = "CIERREN.TTAFOGRAL_IND_CTA_INDV"

db.write_data(db.read_delta(f"DELTA_TRA_MEJORA_INSERT02_{params.sr_folio}"), table_name_CTA_INDV, "default", "append")

In [0]:
#Se elimina la Delta para liberar espacio
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_MEJORA_UPDATE_{params.sr_folio}")
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_MEJORA_INSERT02_{params.sr_folio}")